In [1]:
from pprint import pprint
from collections import Counter
from time import time
from datetime import datetime
import csv
from util import util
from observation import Ilist, Iindex
from copy import copy
import pandas as pd

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/'

In [2]:
file = chemin + 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv'
data = pd.read_csv(file, sep=',')
print('data : \n', len(data), '\n')
#print(data.iloc[0])

data : 
 51276 

nom_amenageur                                                                  LUMI'IN
siren_amenageur                                                            807940069.0
contact_amenageur                                               secretariat@lumi-in.fr
nom_operateur                                                                  LUMI'IN
contact_operateur                                               secretariat@lumi-in.fr
telephone_operateur                                                          754374751
nom_enseigne                                                                   LUMI'IN
id_station_itinerance                                                  FRLUMEACACIAS11
id_station_local                                  ed2da8f2-b28e-11ed-afa1-0242ac120002
nom_station                                    Logis Hôtel des Acacias Lille Tourcoing
implantation_station                                      Parking privé à usage public
adresse_station           

C:\Users\a179227\AppData\Local\Temp\ipykernel_16424\1869881362.py:2: DtypeWarning: Columns (12,23,24,25,26,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, sep=',')


In [14]:
idxs = Ilist.obj(data)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs), len(idxs.idxlen), sum(idxs.idxlen))

idxs (len, lenlidx, sumcodec) :  51276 50 211484


In [15]:
champ = idxs.nindex
print('Couplage entre ', champ('coordonneesXY').name, ' et ', champ('id_station_itinerance').name, ' : ', 
      champ('coordonneesXY').couplinginfos(champ('id_station_itinerance'))['typecoupl'])
infosdefault = champ('id_station_itinerance').couplinginfos(champ('coordonneesXY'), default=True)
print('Ecart : ', infosdefault['disttomin'], 'positions sur ', infosdefault['distmin'], '\n')

Couplage entre  coordonneesXY  et  id_station_itinerance  :  linked
Ecart :  2799 positions sur  18544 



In [16]:
id_station = champ('id_station_itinerance').tostdcodec(full=False)
coordonneesXY = champ('coordonneesXY').tostdcodec(full=False) 

notcoupl = id_station.coupling(coordonneesXY)
c = Counter(id_station.codec).most_common(5)
print('les 5 stations avec le plus de positions: \n', c, '\n')
print('liste des positions associées à la station', c[1][0], ' :')
print(set([coordonneesXY[i] for i in id_station.recordfromvalue(c[1][0])]))

print('nombre de stations en erreur : ', len(set(id_station[notcoupl])))

les 5 stations avec le plus de positions: 
 [('Non concerné', 388), ('FR55CP84000', 20), ('FR55CP83310', 12), ('FRA68P68021001', 10), ('FR073PCAMAIEUFR', 8)] 

liste des positions associées à la station FR55CP84000  :
{'[4.833258, 43.952128]', '[4.876649, 43.925813]', '[4.823253, 43.953885]', '[4.805081, 43.936094]', '[4.813005, 43.928623]', '[4.821979, 43.929502]', '[4.800516, 43.949101]', '[4.82072, 43.935617]', '[4.798628, 43.937354]', '[4.809387, 43.930192]', '[4.828155, 43.947206]', '[4.838556, 43.927832]', '[4.809693, 43.936482]', '[4.799193, 43.929592]', '[4.796913, 43.952673]', '[4.876349, 43.940161]', '[4.825062, 43.942905]', '[4.816939, 43.942256]', '[4.843686, 43.946211]', '[4.821433, 43.939104]'}
nombre de stations en erreur :  2323


In [11]:
notcoupl = champ('coordonneesXY').coupling(champ('nom_station'), derived=False)
print('nombre de pdc avec position/station non couplées : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(100): 
    liste.append((champ('nom_station')[notcoupl[i]], champ('coordonneesXY')[notcoupl[i]]))
pprint(set(liste), width=120)

nombre de pdc avec position/station non couplées :  1718

liste des premières incohérences : 
{(' Intermarché - Retournac - 22kW AC ', '[4.029971, 45.210931]'),
 (' Toyota - Metz- 22kW AC D ', '[2.615871, 48.816567]'),
 ('7 RUE ICARE 67960 ENTZHEIM', '[7.6516, 48.54535]'),
 ('ANTIBES - CASA GENETS - PUBLIC', '[7.030741, 43.624772]'),
 ('Bmw Lattes (PDL 1)', '[3.873362, 43.574581]'),
 ('Bmw Lattes (PDL 2)', '[3.873362, 43.574581]'),
 ('Carrefour  Contact', '[-0.63,44.82]'),
 ('Domaine de Fompeyre', '[-0.63,44.82]'),
 ('Elvi 22kw', '[4.84,45.79]'),
 ('Engen, La Possession Hager', '[55.3292, -20.9426]'),
 ('Engen, La Possession IES', '[55.3292, -20.9426]'),
 ('EvBox DC', '[4.84,45.79]'),
 ('FR*QOV*67-00002', '[7.6516, 48.54535]'),
 ('Fast Automobile Longw', '[5.740974, 49.516179]'),
 ('Fast Automobile Longwy', '[5.740974, 49.516179]'),
 ('Frial_1_RD', '[-0.677194, 49.267985]'),
 ('Frial_1_Siege', '[-0.677194, 49.267985]'),
 ('HAZEBROUCK - Rue François Dominique Depoorter', '[2.534213, 50.

In [6]:
nom_station = champ('nom_station').tostdcodec(full=False)
coordonneesXY = champ('coordonneesXY').tostdcodec(full=False) 
coordonneesXY.coupling(nom_station)
c = Counter(coordonneesXY.codec).most_common(5)
print('les 5 positions avec le plus de stations: \n', c, '\n')
print('liste des stations associées à la position', c[0][0], ' :')
print(set([nom_station[i] for i in coordonneesXY.recordfromvalue(c[0][0])]), '\n')

coordonneesXY = champ('coordonneesXY').tostdcodec(full=False)
nom_station.coupling(coordonneesXY)
c = Counter(nom_station.codec).most_common(5)
print('les 5 stations avec le plus de positions: \n', c, '\n')
print('liste des positions associées à la station', c[0][0], ' :')
print(set([coordonneesXY[i] for i in nom_station.recordfromvalue(c[0][0])]))

les 5 positions avec le plus de stations: 
 [('[2.249932, 48.822635]', 6), ('[3.080477, 50.675889]', 6), ('[2.26765, 48.827423]', 6), ('[2.35813, 48.7467]', 6), ('[3.10733, 50.342696]', 5)] 

liste des stations associées à la position [2.249932, 48.822635]  :
{'25 BOULEVARD DES ILES - ISSY LES MOULINEAUX', '22 BOULEVARD DES ILES - ISSY LES MOULINEAUX', '26 BOULEVARD DES ILES - ISSY LES MOULINEAUX', '24 BOULEVARD DES ILES - ISSY LES MOULINEAUX', '23 BOULEVARD DES ILES - ISSY LES MOULINEAUX', '21 BOULEVARD DES ILES - ISSY LES MOULINEAUX'} 

les 5 stations avec le plus de positions: 
 [('CAMAÏEU FRANCE', 8), ('BAILLARGUES - Gare', 6), ('Saint Andre De Sangonis, Route de Montpellier - 3815', 5), ("Gap, 90 avenue d'Embrun - 3273", 5), ('Camping Arinella', 5)] 

liste des positions associées à la station CAMAÏEU FRANCE  :
{'[3.207187,50.684964]', '[3.20737,50.684898]', '[3.207246,50.68494]', '[3.207306,50.684918]', '[3.206962,50.685049]', '[3.207047,50.685013]', '[3.207124,50.684984]', '[3.2